# Install necessary libraries

In [1]:
!pip install openai gradio
from typing import List, Tuple
import openai
import gradio as gr
from google.colab import userdata

# obtain chatbot API
openai.api_key = userdata.get("ChatGPT")

# Memory and feedback storage
from dataclasses import field
import pandas as pd
import csv
import json
import os
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

# Memory and feedback storage to CSV

In [2]:
## future implementation, too complicated for this moment. using json cache for now

# Assignment Requirements w/Cache

In [3]:
# requirements summary cache (to avoid multiple API calls)
REQUIREMENTS_CACHE_FILE = "requirements_cache.json"

def load_cache():
  """ Load the requirements cache from a file """
  if os.path.exists(REQUIREMENTS_CACHE_FILE):
    with open(REQUIREMENTS_CACHE_FILE, "r") as file:
      return json.load(file)
  return {}

def save_cache(cache):
  """Save the requirements cache to a file"""
  with open(REQUIREMENTS_CACHE_FILE, "w") as file:
    json.dump(cache, file)

# retrieve or create the requirements
def get_or_store_requirements(assignment_id: str, requirements_text: str):
  """Retrieve a cached summary or generate a new one if missing"""

  #validate if assigment_id is given
  if not assignment_id.strip():
    return "**Error:** Missing assignment ID. Please enter a valid assignment ID."

  cache = load_cache()
  cache_key = f"{assignment_id}_requirements"

  # Return cached requirements if available
  if cache_key in cache and cache[cache_key]:
    return cache[cache_key]

  # Ensure the requirements text is not empty
  if not requirements_text.strip():
    return "--No assignment requirements provided."

  # Store it in the cache
  cache[cache_key] = requirements_text
  save_cache(cache)

  return requirements_text  # Returns the requirements

# Generate Inline Feedback

In [4]:
# Feedback Class to store inline feedback
class Feedback:
  def __init__(self, excerpt: str, feedback: str, category: str):
    self.excerpt = excerpt
    self.feedback = feedback
    self.category = category

  def to_dict(self):
    return {
        "excerpt": self.excerpt,
        "feedback": self.feedback,
        "category": self.category
    }

# Generate inline feedback
def generate_inline_feedback(assignment_id: str, submission_text: str, requirements_text: str):
  """
  Processes the full essay, compares it to assignment requirements, and provides inline feedback.
  Uses summarized assignment requirements to reduce token usage.
  Returns 3-5 inline feedback suggestions structured JSON feedback targeting specific areas for improvement.

  """

  #check if assignment_id is empty to check cache
  if not assignment_id.strip():
    return "**Error:** Missing assignment ID. Please enter a valid assignment ID."

  #check if submission is empty
  if not submission_text.strip():
    return "**Error:** No submission provided. Please enter a student submission."

  requirements_submitted = get_or_store_requirements(assignment_id, requirements_text)

  prompt = f"""
    Instructions:
    You are an expert writing tutor providing constructive inline feedback on a student's essay.
    - Analyze the student's assignment while considering the provided **requirements**.
    - Provide **3-5 inline feedback points** in JSON format, ensuring feedback aligns with grading expectations.
    - STRICTLY RETURN ONLY JSON OUTPUT

    ### Requirements Summary:
    {requirements_submitted}

    ### Student Assignment:
    {submission_text}

    **Format for Response (JSON Only):**
    ```json
    [
        {{"excerpt": "Some part of the student's text",
        "feedback": "Your feedback here",
        "category": "Grammar/Clarity/Argument/Structure/Requirements"}}
    ]
    ```

    **Generated Feedback (JSON Format):**
  """

  response = openai.chat.completions.create(
      model="gpt-4o",
      messages=[{"role": "user", "content": prompt}],  # Send entire history as messages
      temperature=0.7,  # Increased randomness has given better results for feedback
      max_tokens=1000,
  )

  response_text = response.choices[0].message.content.strip()

  # Handle empty response
  if not response_text:
    return "No inline feedback generated. Try again."

  # Extract JSON if response contains extra text
  match = re.search(r"\[\s*{.*?}\s*\]", response_text, re.DOTALL)
  if match:
    response_text = match.group(0)

  #attempt to parse response as JSON
  try:
    feedback_list = json.loads(response_text)
    formatted_feedback = "\n".join([
      f"• Excerpt: \"{item['excerpt']}\"\n"
      f"• Feedback: {item['feedback']}\n"
      f"• Category: {item['category']}\n"
      "-------------------------------------"
      for item in feedback_list
    ])
  except (json.JSONDecodeError, TypeError, KeyError) as e:
    formatted_feedback = "--Error parsing JSON."

  return formatted_feedback  # This returns properly formatted feedback





# Generate Summary Feedback

In [5]:
def generate_summary_feedback(assignment_id: str, submission_text: str, requirements_text: str):
  """
  Generates high-level summary feedback on the entire essay, considering assignment requirements.
  Returns structured feedback on strengths, weaknesses, next steps, and rubric alignment.
  """
  #check if assignment_id is empty to check cache
  if not assignment_id.strip():
    return "**Error:** Missing assignment ID. Please enter a valid assignment ID."

  #check if submission is empty
  if not submission_text.strip():
    return "**Error:** No submission provided. Please enter a student submission."

  #retrieve or make requirements summary
  requirements_submitted = get_or_store_requirements(assignment_id, requirements_text)

  prompt = f"""
  Instructions:
  You are an expert writing turor proing constructive inline feedback on a student essay.
  - Analyze the student's assignment while considering the provided **requirements**.
  - Provide an **overall summary of the feedback** with the following structure:
    - A brief paragraph on strengths.
    - A brief paragraph on weaknesses.
    - Two to three actionable next steps for improvements.
    - A rubric alignment score from 1 to 10.

  Format the response as:
  **Strengths:** ...
  **Weaknesses:** ...
  **Next Steps:** ...
  **Rubric Alignment:** X/10

  ### Assignment Requirements:
  {requirements_submitted}

  ### Student's Essay:
  {submission_text}

  **Generated Summary Feedback:**
  """
  response = openai.chat.completions.create(
      model="gpt-4o",
      messages=[{"role": "user", "content": prompt}],
      temperature=0.7,  # Balance randomness for useful feedback?
      max_tokens=500
  )

  summary_feedback = response.choices[0].message.content.strip()

  return summary_feedback

# Gradio UI Integration

In [ ]:
# Create Gradio UI
def gradio_ui():
  with gr.Blocks() as demo:
    gr.Markdown("# AI Feedback Generator")

    # Input fields
    student_id_input = gr.Textbox(label="Student ID")
    assignment_id_input = gr.Textbox(label="Assignment ID")
    assignment_input = gr.Textbox(label="Assignment Requirements", lines=5, scale=2)
    submission_input = gr.Textbox(label="Full Student Essay", lines=5, scale=2)

    #Action buttons
    with gr.Row():
      inline_button = gr.Button(value="Generate Inline Feedback")
      summary_button = gr.Button(value="Generate Summary Feedback")

    #output fields
    inline_output = gr.Textbox(label="Inline Feedback", lines=5)
    summary_output = gr.Textbox(label="Summary Feedback", lines=5)

    #button actions
    inline_button.click(
      generate_inline_feedback,
      inputs=[assignment_id_input, submission_input, assignment_input],
      outputs=[inline_output]
    )
    summary_button.click(
      generate_summary_feedback,
      inputs=[assignment_id_input, submission_input, assignment_input],
      outputs=[summary_output]
    )

  return demo

#launch UI
gradio_ui().launch(debug=True, share=True, height=1000)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b0b2a4a6a717e29cad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
